# Persian Sentiment
It aims to classify text, such as comments, based on their emotional bias.

In [ ]:
!nvidia-smi
!lscpu

In [2]:
!pip install hazm==0.7.0
!pip install seqeval==1.2.2
!pip install sentencepiece==0.1.96
!pip install transformers==4.7.0
!pip install clean-text[gpl]==0.4.0

     |████████████████████████████████| 316 kB 8.6 MB/s 
     |████████████████████████████████| 233 kB 37.2 MB/s 
     |████████████████████████████████| 1.4 MB 45.4 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=98c1d516ab0bd8d9d98dfaa6faf951dd2f95649d9fb11a20cd1a1860835c4b78
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154539 sha256=2bde2413e96de00c4c223721d6e9684c59abc34430693cea223856c123e1d55c
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 43 kB 1.5 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.

In [3]:
!pip install PyDrive
import os
import IPython.display as ipd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# Import required packages
import os
import gc
import re
import hazm
import time
import json
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import transformers
from transformers import AutoConfig, AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import MT5Config, MT5ForConditionalGeneration, MT5Tokenizer

from cleantext import clean

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print()
print('numpy', np.__version__)
print('pandas', pd.__version__)
print('transformers', transformers.__version__)
print('torch', torch.__version__)
print()

# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")



numpy 1.19.5
pandas 1.1.5
transformers 4.7.0
torch 1.9.0+cu102

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [5]:
class SentimentAnalysisDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Sentiment Analysis. """

    def __init__(self, tokenizer, comments, targets, label_list=None, max_len=128):
        self.comments = comments
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.label2index = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
        self.index2label = {i: label for label, i in self.label2index.items()}

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, item):
        comment = self.comments[item]
        target = self.label2index[self.targets[item]]
        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt')

        inputs = {
            'comment': comment,
            'targets': torch.tensor(target, dtype=torch.long),
            'original_targets': self.targets[item],
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        return inputs


class MT5SentimentAnalysisDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset for Sentiment Analysis. """

    def __init__(self, reviews, aspects, labels, tokenizer, max_length=128):
        self.reviews = reviews
        self.aspects = aspects
        self.targets = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        if self.aspects is not None:
            encoding = self.tokenizer(
                self.reviews[item] + " <sep> " + self.aspects[item],
                add_special_tokens=True,
                max_length=self.max_length,
                truncation=True,
                padding='max_length',
                return_tensors="pt"
            )
            inputs = {
                'review': self.reviews[item],
                'aspects': self.aspects[item],
                'targets': self.targets[item],
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten()
            }
        else:
            encoding = self.tokenizer(
                self.reviews[item],
                add_special_tokens=True,
                max_length=self.max_length,
                truncation=True,
                padding='max_length',
                return_tensors="pt"
            )
            inputs = {
                'review': self.reviews[item],
                'targets': self.targets[item],
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten()
            }

        return inputs


class SentimentAnalysis:
    def __init__(self, model_name, model_type=None):
        self.normalizer = hazm.Normalizer()
        self.model_name = model_name
        if model_type == "mt5":
            self.tokenizer = MT5Tokenizer.from_pretrained(model_name)
            self.model = MT5ForConditionalGeneration.from_pretrained(model_name)
            self.config = MT5Config.from_pretrained(self.model_name)
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForSequenceClassification.from_pretrained(self.model_name)
            self.config = AutoConfig.from_pretrained(self.model_name)
            self.id2label = self.config.id2label
            self.label2id = self.config.label2id

    def cleaning(self, text):
        def cleanhtml(raw_html):
            clean_pattern = re.compile('<.*?>')
            clean_text = re.sub(clean_pattern, '', raw_html)
            return clean_text

        if type(text) is not str:
            return None

        text = text.strip()

        # regular cleaning
        text = clean(
            text,
            fix_unicode=True,
            to_ascii=False,
            lower=True,
            no_line_breaks=True,
            no_urls=True,
            no_emails=True,
            no_phone_numbers=True,
            no_numbers=False,
            no_digits=False,
            no_currency_symbols=True,
            no_punct=False,
            replace_with_url="",
            replace_with_email="",
            replace_with_phone_number="",
            replace_with_number="",
            replace_with_digit="0",
            replace_with_currency_symbol=""
        )

        # cleaning htmls
        text = cleanhtml(text)

        # normalizing
        text = self.normalizer.normalize(text)

        # removing wierd patterns
        wierd_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u'\U00010000-\U0010ffff'
                                   u"\u200d"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\u3030"
                                   u"\ufe0f"
                                   u"\u2069"
                                   u"\u2066"
                                   # u"\u200c"
                                   u"\u2068"
                                   u"\u2067"
                                   "]+", flags=re.UNICODE)

        text = wierd_pattern.sub(r'', text)

        # removing extra spaces, hashtags
        text = re.sub("#", "", text)
        text = re.sub("\s+", " ", text)
        if text in ['', " "]:
            return None
        return text

    def load_dataset_test_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() == "snappfood":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_csv(dataset_file, delimiter="\t")
            # drop label_id because its not consistent with albert model labels!
            data = data[['comment', 'label']]

            # cleaning comments
            data = data.dropna(subset=['comment'])
            data['comment'] = data['comment'].apply(self.cleaning)
            data = data.dropna(subset=['comment'])

            if 'label_map' in kwargs:
                data['label'] = data['label'].apply(lambda l: kwargs['label_map'][l])
                data = data.dropna(subset=['label'])
                data = data.reset_index(drop=True)

            data['label_id'] = data['label'].apply(lambda t: self.label2id[t])
            x_test, y_test = data['comment'].values.tolist(), data['label_id'].values.tolist()
            print(f'test part:\n #comment: {len(x_test)}, #labels: {len(y_test)}')
            return x_test, y_test
        if dataset_name.lower() == "deepsentipers":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            if 'label_map' not in kwargs:
                print("label_map is missing!")
                return
            data = pd.read_csv(dataset_file, delimiter=",", names=['comment', 'label'], header=None)

            # cleaning comments
            data = data.dropna(subset=['comment'])
            data['comment'] = data['comment'].apply(self.cleaning)
            data = data.dropna(subset=['comment'])

            # map labels
            label_map = kwargs['label_map']
            data['label'] = data['label'].apply(lambda l: label_map[l])
            data = data.dropna(subset=['label'])
            data = data.reset_index(drop=True)

            data['label_id'] = data['label'].apply(lambda t: self.label2id[t])
            x_test, y_test = data['comment'].values.tolist(), data['label_id'].values.tolist()
            print(f'test part:\n #comment: {len(x_test)}, #labels: {len(y_test)}')
            return x_test, y_test
        if dataset_name.lower() == "pasinlu-aspect-sentiment":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            if 'label_map' not in kwargs:
                print("label_map is missing!")
                return

            reviews, aspects, labels = [], [], []
            with open(dataset_file, encoding="utf8") as infile:
                for line in infile:
                    json_line = json.loads(line.strip())

                    review = json_line['review']
                    reviews.append(review)

                    question = json_line['question']
                    aspects.append(question)

                    label = kwargs['label_map'][json_line['label']]
                    labels.append(label)

            return reviews, aspects, labels

    def load_dataset_file(self, dataset_name, dataset_file, **kwargs):
        if dataset_name.lower() == "digikala":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            data = pd.read_excel(dataset_file)
            data = data[['comment', 'recommend']]

            # cleaning comments
            data = data.dropna(subset=['comment'])
            data['comment'] = data['comment'].apply(self.cleaning)
            data = data.dropna(subset=['comment'])

            # cleaning labels
            valid_labels = ['no_idea', 'not_recommended', 'recommended']
            data['recommend'] = data['recommend'].apply(lambda r: r if r in valid_labels else None)
            data = data.dropna(subset=['recommend'])
            if 'label_map' in kwargs:
                data['recommend'] = data['recommend'].apply(lambda l: kwargs['label_map'][l])
            data = data.dropna(subset=['recommend'])
            data = data.reset_index(drop=True)

            data['label_id'] = data['recommend'].apply(lambda t: self.label2id[t])

            x_all, y_all = data['comment'].values.tolist(), data['label_id'].values.tolist()
            print(f'all data: #comment: {len(x_all)}, #labels: {len(y_all)}')

            _, test = train_test_split(data, test_size=0.1, random_state=1, stratify=data['recommend'])
            test = test.reset_index(drop=True)
            x_test, y_test = test['comment'].values.tolist(), test['label_id'].values.tolist()
            print(f'test part:\n #comment: {len(x_test)}, #labels: {len(y_test)}')
            return x_all, y_all, x_test, y_test
        if dataset_name.lower() == "pasinlu-review-sentiment":
            if not os.path.exists(dataset_file):
                print(f'{dataset_file} not exists!')
                return
            if 'label_map' not in kwargs:
                print("label_map is missing!")
                return

            reviews, labels = [], []
            with open(dataset_file, encoding="utf8") as infile:
                for line in infile:
                    json_line = json.loads(line.strip())

                    review = json_line['review']
                    reviews.append(review)

                    label = kwargs['label_map'][json_line['sentiment']]
                    labels.append(label)
            return reviews, labels

    def load_dataset_composite_file(self, dataset_name, dataset_files, **kwargs):
        if dataset_name.lower() == "digikala+snappfood+deepsentipers":
            if sorted(list(dataset_files.keys())) != ["deepsentipers", "digikala", "snappfood"]:
                print("dataset_files must contains path of all three datasets")
                return
            if 'label_map' not in kwargs:
                print("label_map is missing!")
                return
            elif sorted(list(kwargs['label_map'].keys())) != ["deepsentipers", "digikala", "snappfood"]:
                print("label_map must contains label_map for all three datasets!")
                return
            print("digikala dataset - we only use test set:")
            _, _, x_test_digi, y_test_digi = self.load_dataset_file('digikala', dataset_files['digikala'],
                                                                    label_map=kwargs['label_map']['digikala'])
            print("snappfood dataset:")
            x_test_snapp, y_test_snapp = self.load_dataset_test_file('snappfood', dataset_files['snappfood'],
                                                                     label_map=kwargs['label_map']['snappfood'])
            print("deepsentipers dataset:")
            x_test_senti, y_test_senti = self.load_dataset_test_file('deepsentipers', dataset_files['deepsentipers'],
                                                                     label_map=kwargs['label_map']['deepsentipers'])
            return x_test_digi + x_test_snapp + x_test_senti, y_test_digi + y_test_snapp + y_test_senti

    def sentiment_analysis_inference(self, input_text, device):
        if not self.model or not self.tokenizer or not self.id2label:
            print('Something wrong has been happened!')
            return

        pt_batch = self.tokenizer(
            input_text,
            padding=True,
            truncation=True,
            max_length=self.config.max_position_embeddings,
            return_tensors="pt"
        )

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()
        pt_batch = pt_batch.to(device)

        pt_outputs = self.model(**pt_batch)
        pt_predictions = torch.argmax(F.softmax(pt_outputs.logits, dim=1), dim=1)

        output_predictions = []
        for i, sentence in enumerate(input_text):
            output_predictions.append((sentence, self.id2label.get(pt_predictions[i].item())))
        return output_predictions

    def mt5_sentiment_analysis_inference(self, reviews, device):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return

        tokenized_batch = self.tokenizer(
            reviews,
            padding=True,
            return_tensors="pt"
        )

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        input_ids = tokenized_batch.input_ids.to(device)
        attention_mask = tokenized_batch.attention_mask.to(device)
        outputs = self.model.generate(input_ids=input_ids,
                                      attention_mask=attention_mask)
        predictions = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return predictions

    def mt5_aspect_sentiment_analysis_inference(self, reviews, aspects, device):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return

        new_input = []
        for r, a in zip(reviews, aspects):
            new_input.append(r + " <sep> " + a)

        tokenized_batch = self.tokenizer(
            new_input,
            padding=True,
            return_tensors="pt"
        )

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        input_ids = tokenized_batch.input_ids.to(device)
        attention_mask = tokenized_batch.attention_mask.to(device)
        outputs = self.model.generate(input_ids=input_ids,
                                      attention_mask=attention_mask)
        predictions = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        return predictions

    def evaluation(self, input_text, input_labels, device, batch_size=4):
        if not self.model or not self.tokenizer or not self.id2label:
            print('Something wrong has been happened!')
            return

        max_len = self.config.max_position_embeddings
        label_list = list(set(input_labels))
        label_count = {self.id2label[label]: input_labels.count(label) for label in label_list}
        print("label_count:", label_count)
        dataset = SentimentAnalysisDataset(comments=input_text, targets=input_labels, tokenizer=self.tokenizer,
                                           max_len=max_len, label_list=label_list)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

        print("#samples:", len(input_text))
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        total_loss, total_time = 0, 0
        output_predictions = []
        golden_labels, predicted_labels = [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            b_comments = batch['comment']
            b_input_ids = batch['input_ids']
            b_attention_mask = batch['attention_mask']
            b_token_type_ids = batch['token_type_ids']
            b_targets = batch['targets']

            # move tensors to GPU if CUDA is available
            b_input_ids = b_input_ids.to(device)
            b_attention_mask = b_attention_mask.to(device)
            b_token_type_ids = b_token_type_ids.to(device)
            b_targets = b_targets.to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model(input_ids=b_input_ids, attention_mask=b_attention_mask,
                                       token_type_ids=b_token_type_ids, labels=b_targets)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')
            # get the loss
            total_loss += b_outputs.loss.item()

            b_original_targets = batch['original_targets']
            golden_labels.extend(b_original_targets.tolist())

            b_predictions = torch.argmax(F.softmax(b_outputs.logits, dim=1), dim=1)
            b_predictions = b_predictions.cpu().detach().numpy().tolist()
            b_predictions = [dataset.index2label[label] for label in b_predictions]
            predicted_labels.extend(b_predictions)

            for i, comment in enumerate(b_comments):
                output_predictions.append((
                    comment,
                    self.id2label[b_original_targets[i].item()],
                    self.id2label[b_predictions[i]]
                ))
                # print(f'output prediction: {i},{comment},{self.id2label[b_original_targets[i].item()]},'
                #       f'{self.id2label[b_predictions[i]]}')

        # Calculate the average loss over the training data.
        avg_train_loss = total_loss / len(data_loader)
        print("average loss:", avg_train_loss)
        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(input_text))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_labels, predicted_labels)))
        print("Test Precision: {}".format(precision_score(golden_labels, predicted_labels, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_labels, predicted_labels, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_labels, predicted_labels, average="weighted")))
        print("Test classification Report:\n{}".format(classification_report(
            golden_labels, predicted_labels, digits=10, target_names=[self.id2label[_] for _ in sorted(label_list)])))
        return output_predictions

    def mt5_sentiment_analysis_evaluation(self, reviews, labels, device, max_length, batch_size=4):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return
        if len(reviews) != len(labels):
            print('length of inputs and labels is not equal!!')
            return

        dataset = MT5SentimentAnalysisDataset(reviews=reviews, aspects=None, labels=labels, tokenizer=self.tokenizer,
                                              max_length=max_length)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#reviews:{len(reviews)}, #labels:{len(labels)}')
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        total_time = 0
        output_predictions = []
        golden_labels, predicted_labels = [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            # move tensors to GPU if CUDA is available
            b_input_ids = batch['input_ids'].to(device)
            b_attention_mask = batch['attention_mask'].to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model.generate(input_ids=b_input_ids, attention_mask=b_attention_mask)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')

            b_targets = batch['targets']
            golden_labels.extend(b_targets)

            b_predictions = self.tokenizer.batch_decode(b_outputs, skip_special_tokens=True)
            predicted_labels.extend(b_predictions)

            for i, review in enumerate(batch['review']):
                output_predictions.append((
                    review,
                    b_targets[i],
                    b_predictions[i]
                ))

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(reviews))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_labels, predicted_labels)))
        print("Test Precision: {}".format(precision_score(golden_labels, predicted_labels, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_labels, predicted_labels, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_labels, predicted_labels, average="weighted")))
        print("Test classification Report:\n{}".format(
            classification_report(golden_labels, predicted_labels, digits=10)))
        return output_predictions

    def mt5_aspect_sentiment_analysis_evaluation(self, reviews, aspects, labels, device, max_length, batch_size=4):
        if not self.model or not self.tokenizer:
            print('Something wrong has been happened!')
            return
        if len(reviews) != len(labels):
            print('length of inputs and labels is not equal!!')
            return

        dataset = MT5SentimentAnalysisDataset(reviews=reviews, aspects=aspects, labels=labels, tokenizer=self.tokenizer,
                                              max_length=max_length)
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
        print(f'#reviews:{len(reviews)}, #aspects:{len(aspects)}, #labels:{len(labels)}')
        print("#batch:", len(data_loader))

        gc.collect()
        torch.cuda.empty_cache()
        # Tell pytorch to run this model on the GPU.
        if device.type != 'cpu':
            self.model.cuda()
        self.model.eval()

        total_time = 0
        output_predictions = []
        golden_labels, predicted_labels = [], []
        print("Start to evaluate test data ...")
        for step, batch in enumerate(data_loader):
            # move tensors to GPU if CUDA is available
            b_input_ids = batch['input_ids'].to(device)
            b_attention_mask = batch['attention_mask'].to(device)

            # This will return the loss (rather than the model output) because we have provided the `labels`.
            with torch.no_grad():
                start = time.monotonic()
                b_outputs = self.model.generate(input_ids=b_input_ids, attention_mask=b_attention_mask)
                end = time.monotonic()
                total_time += end - start
                print(f'inference time for step {step}: {end - start}')

            b_targets = batch['targets']
            golden_labels.extend(b_targets)

            b_predictions = self.tokenizer.batch_decode(b_outputs, skip_special_tokens=True)
            predicted_labels.extend(b_predictions)

            for i, review in enumerate(batch['review']):
                output_predictions.append((
                    review,
                    batch['aspects'][i],
                    b_targets[i],
                    b_predictions[i]
                ))

        print("total inference time:", total_time)
        print("total inference time / #samples:", total_time / len(reviews))

        # evaluate
        print("Test Accuracy: {}".format(accuracy_score(golden_labels, predicted_labels)))
        print("Test Precision: {}".format(precision_score(golden_labels, predicted_labels, average="weighted")))
        print("Test Recall: {}".format(recall_score(golden_labels, predicted_labels, average="weighted")))
        print("Test F1-Score(weighted average): {}".format(
            f1_score(golden_labels, predicted_labels, average="weighted")))
        print("Test classification Report:\n{}".format(
            classification_report(golden_labels, predicted_labels, digits=10)))
        return output_predictions


In [6]:
model_name='persiannlp/mt5-large-parsinlu-sentiment-analysis'
sa_model = SentimentAnalysis(model_name, model_type="mt5")
print(sa_model.config)


MT5Config {
  "_name_or_path": "/home/patrick/hugging_face/t5/mt5-large",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 2816,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.7.0",
  "use_cache": true,
  "vocab_size": 250112
}



## Sample Inference:

In [7]:
review_list = [
   "یک فیلم ضعیف بی محتوا بدون فیلمنامه . شوخی های سخیف .",
    "فیلم تا وسط فیلم یعنی دقیقا تا جایی که معلوم میشه بچه های املشی دنبال رضان خیلی خوب و جذاب پیش میره ولی دقیقا از همونجاش سکته میزنه و خلاص...",
    "اصلا به هیچ عنوان علاقه نداشتم اجرای می سی سی پی نشسته میمیرد روی پرده سینما ببینم  دیالوگ های تکراری   هلیکوپتر  ماشین  آلندلون  لئون  پاپیون  آخه چرااااااااااااااا   همون حسی که توی تالار وحدت بعد از نیم ساعت به سرم اومد امشب توی سالن سینما تجربه کردم ،حس گریز از سالن.......⁦ ⁦(ノಠ益ಠ)ノ⁩ ",
    " گول نخورید این رنگارنگ مینو نیست برای شرکت گرجیه و متاسفانه این محصولش اصلا مزه رنگارنگی که انتظار دارید رو نمیده ",
    "در مقایسه با سایر برندهای موجود در بازار با توجه به حراجی که داشت ارزانتر ب",
    "من پسرم عاشق ایناس ولی دیگه به خاطر حفظ محیط زیست فقط زمانهایی که مجبور باشم شیر دونه ای میخرم و سعی میکنم دیگه کمتر شیر با بسته بندی تتراپک استفاده کنم ",
]
aspect_list = [
    "نظر شما در مورد داستان، فیلمنامه، دیالوگ ها و موضوع فیلم  لونه زنبور چیست؟",
    "نظر شما به صورت کلی در مورد فیلم  ژن خوک چیست؟",
    " نظر شما در مورد صداگذاری و جلوه های صوتی فیلم  مسخره‌باز چیست؟",
    " نظر شما در مورد عطر، بو، و طعم این بیسکویت و ویفر چیست؟",
    " شما در مورد قیمت و ارزش خرید این حبوبات و سویا چیست؟",
    "نظر شما به صورت کلی در مورد این شیر چیست؟"
]

In [8]:
sa_model.mt5_sentiment_analysis_inference(review_list, device)

['very negative',
 'mixed',
 'no sentiment expressed',
 'very negative',
 'positive',
 'no sentiment expressed']

In [9]:
sa_model.mt5_aspect_sentiment_analysis_inference(review_list, aspect_list, device)

['negative',
 'mixed',
 'no sentiment expressed',
 'very negative',
 'positive',
 'mixed']

## ParsiNLU Sentiment Analysis


In [10]:
!git clone https://github.com/persiannlp/parsinlu
!ls parsinlu
!ls parsinlu/data/sentiment-analysis/

Cloning into 'parsinlu'...
remote: Enumerating objects: 1434, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 1434 (delta 110), reused 139 (delta 82), pack-reused 1252
Receiving objects: 100% (1434/1434), 27.81 MiB | 19.15 MiB/s, done.
Resolving deltas: 100% (913/913), done.
data  LICENSE  README.md  requirements.txt  scripts  src
ABSA_Dataset_train.jsonl  human_evaluation_food.jsonl	     movie.jsonl
food_dev.jsonl		  human_evaluation_food_test.jsonl   movie_test.jsonl
food.jsonl		  human_evaluation_movie.jsonl	     movie_train.jsonl
food_test.jsonl		  human_evaluation_movie_test.jsonl
food_train.jsonl	  movie_dev.jsonl


### Aspect-base Sentiment Analysis - food category

In [11]:
test_reviews_food, test_aspects_food, test_labels_food = sa_model.load_dataset_test_file(
    dataset_name="pasinlu-aspect-sentiment", 
    dataset_file="./parsinlu/data/sentiment-analysis/food_test.jsonl",
    label_map={
        '-3': 'no sentiment expressed',
        '-2': 'very negative',
        '-1': 'negative',
        '0': 'neutral',
        '1': 'positive',
        '2': 'very positive',
        '3': 'mixed'
        }
    )
print(test_reviews_food[:5])
print(test_aspects_food[:5])
print(test_labels_food[:5])
print(len(test_reviews_food))
print(len(test_aspects_food))
print(len(test_labels_food))

['مدت هاست که از محصول همین برند استفاده می کنم. کیفیت کاملا قابل قبولی داره\u200c و قیمتش مناسبه.', 'مدت هاست که از محصول همین برند استفاده می کنم. کیفیت کاملا قابل قبولی داره\u200c و قیمتش مناسبه.', 'مدت هاست که از محصول همین برند استفاده می کنم. کیفیت کاملا قابل قبولی داره\u200c و قیمتش مناسبه.', 'مدت هاست که از محصول همین برند استفاده می کنم. کیفیت کاملا قابل قبولی داره\u200c و قیمتش مناسبه.', 'مدت هاست که از محصول همین برند استفاده می کنم. کیفیت کاملا قابل قبولی داره\u200c و قیمتش مناسبه.']
['نظر شما در مورد عطر، بو، و طعم این قهوه چیست؟', 'نظر شما در مورد قیمت و ارزش خرید این قهوه چیست؟', 'نظر شما در مورد ارسال و حمل و نقل این قهوه چیست؟', 'نظر شما در مورد بسته بندی و نگهداری این قهوه چیست؟', 'نظر شما در مورد سلامت و ارزش غذایی این قهوه چیست؟']
['no sentiment expressed', 'positive', 'no sentiment expressed', 'no sentiment expressed', 'no sentiment expressed']
1344
1344
1344


In [12]:
!nvidia-smi
!lscpu

Mon Aug  2 13:03:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W /  70W |   6144MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
sa_model.mt5_aspect_sentiment_analysis_inference(test_reviews_food[:5], test_aspects_food[:5], device)

['no sentiment expressed',
 'positive',
 'no sentiment expressed',
 'no sentiment expressed',
 'no sentiment expressed']

In [14]:
evaluation_output_as_food = sa_model.mt5_aspect_sentiment_analysis_evaluation(test_reviews_food, test_aspects_food, test_labels_food, device, max_length=512, batch_size=32)

#reviews:1344, #aspects:1344, #labels:1344
#batch: 42
Start to evaluate test data ...
inference time for step 0: 4.530266835999953
inference time for step 1: 4.546181868000019
inference time for step 2: 4.577880920000041
inference time for step 3: 4.59797387499998
inference time for step 4: 4.616710951000016
inference time for step 5: 4.60532237000001
inference time for step 6: 4.648039934999929
inference time for step 7: 4.65350778100003
inference time for step 8: 4.675366069000006
inference time for step 9: 4.6868966130000445
inference time for step 10: 4.700032532000023
inference time for step 11: 4.713500253999996
inference time for step 12: 4.728824198999973
inference time for step 13: 4.729174860000057
inference time for step 14: 4.743795540000065
inference time for step 15: 4.760375643999964
inference time for step 16: 4.752080189000026
inference time for step 17: 4.7750269329999355
inference time for step 18: 4.779700998999942
inference time for step 19: 4.789267754999969
infer

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
for review, aspect, true_label, predicted_label in evaluation_output_as_food[:25]:
  print('{}\t{}\t{}\t{}'.format(review, aspect, true_label, predicted_label))

مدت هاست که از محصول همین برند استفاده می کنم. کیفیت کاملا قابل قبولی داره‌ و قیمتش مناسبه.	نظر شما در مورد عطر، بو، و طعم این قهوه چیست؟	no sentiment expressed	no sentiment expressed
مدت هاست که از محصول همین برند استفاده می کنم. کیفیت کاملا قابل قبولی داره‌ و قیمتش مناسبه.	نظر شما در مورد قیمت و ارزش خرید این قهوه چیست؟	positive	positive
مدت هاست که از محصول همین برند استفاده می کنم. کیفیت کاملا قابل قبولی داره‌ و قیمتش مناسبه.	نظر شما در مورد ارسال و حمل و نقل این قهوه چیست؟	no sentiment expressed	no sentiment expressed
مدت هاست که از محصول همین برند استفاده می کنم. کیفیت کاملا قابل قبولی داره‌ و قیمتش مناسبه.	نظر شما در مورد بسته بندی و نگهداری این قهوه چیست؟	no sentiment expressed	no sentiment expressed
مدت هاست که از محصول همین برند استفاده می کنم. کیفیت کاملا قابل قبولی داره‌ و قیمتش مناسبه.	نظر شما در مورد سلامت و ارزش غذایی این قهوه چیست؟	no sentiment expressed	no sentiment expressed
مدت هاست که از محصول همین برند استفاده می کنم. کیفیت کاملا قابل قبولی داره‌ و قیمتش مناسبه.	نظ

In [16]:
output_file_name = "aspect_sentiment_analysis_food_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for review, aspect, true_label, predicted_label in evaluation_output_as_food:
    output_file.write('{}\t{}\t{}\t{}\n'.format(review, aspect, true_label, predicted_label))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Sentiment Analysis - food category

In [17]:
all_reviews_food, all_labels_food = sa_model.load_dataset_file(
    dataset_name="pasinlu-review-sentiment", 
    dataset_file="./parsinlu/data/sentiment-analysis/food.jsonl",
    label_map={
        '-3': 'no sentiment expressed',
        '-2': 'very negative',
        '-1': 'negative',
        '0': 'neutral',
        '1': 'positive',
        '2': 'very positive',
        '3': 'mixed'
        }
    )
print(all_reviews_food[:5])
print(all_labels_food[:5])
print(len(all_reviews_food))
print(len(all_labels_food))

['من یه مدته فقط از این محصول استفاده میکنم اما جدیدا دونه هاش درشت شده،و روی خود محصول هم نوشته تراریخته.', 'اگه بخوای به صورت مکمل استفاده اش کنی خوبه وارزش استفاده روداره.', 'طعم خیلی خوبی داره و توضیحات کالا کاملا با خود محصول مطابقت داشت', 'من این رو که گرفتم خیلی به نظر مونده بود و بشدت بوی بد میداد . حتی رنگش هم کمی عوض شده بود که مجبور شدم هنش رو دور بندازم', 'کلا که نوشابه محصول مفیدی نیست. اما اونایی که استفاده دارند توی شگفت قیمتش خوبه.']
['mixed', 'positive', 'very positive', 'very negative', 'positive']
1917
1917


just keep test samples:

In [18]:
test_subset = []
for i, review in enumerate(all_reviews_food):
  if review in test_reviews_food:
    test_subset.append((review, all_labels_food[i]))
all_reviews_food, all_labels_food = [t[0] for t in test_subset], [t[1] for t in test_subset]
print(all_reviews_food[:5])
print(all_labels_food[:5])
print(len(all_reviews_food))
print(len(all_labels_food))

['طعم خیلی خوبی داره و توضیحات کالا کاملا با خود محصول مطابقت داشت', 'خوش طعم بود من طعم کارامل حس نکردم سرد مصرف کنید گرمش خوشمزه نیست', 'من پسرم عاشق ایناس ولی دیگه به خاطر حفظ محیط زیست فقط زمانهایی که مجبور باشم شیر دونه ای میخرم و سعی میکنم دیگه کمتر شیر با بسته بندی تتراپک استفاده کنم', 'این مدلو هم خودمون دوست داریم هم هامو کوچولوها, ظاهرش جالبه و خیلی هم توخالی نیست', 'در شگفت انگیز گرفتم قیمت مناسبی داشت امیدوارم دیر پز نباشه']
['very positive', 'positive', 'mixed', 'very positive', 'positive']
192
192


In [19]:
!nvidia-smi
!lscpu

Mon Aug  2 13:06:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    43W /  70W |   9238MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [20]:
sa_model.mt5_sentiment_analysis_inference(all_reviews_food[:5], device)

['very positive', 'negative', 'no sentiment expressed', 'positive', 'positive']

In [21]:
evaluation_output_s_food = sa_model.mt5_sentiment_analysis_evaluation(all_reviews_food, all_labels_food, device, max_length=512, batch_size=32)

#reviews:192, #labels:192
#batch: 6
Start to evaluate test data ...
inference time for step 0: 4.930177381000021
inference time for step 1: 4.920810561999929
inference time for step 2: 4.920128996000017
inference time for step 3: 4.937980627999991
inference time for step 4: 4.93157474599991
inference time for step 5: 4.914572869999915
total inference time: 29.555245182999784
total inference time / #samples: 0.15393356866145722
Test Accuracy: 0.6927083333333334
Test Precision: 0.7609225109626264
Test Recall: 0.6927083333333334
Test F1-Score(weighted average): 0.7181637624876601
Test classification Report:
                        precision    recall  f1-score   support

                 mixed  0.7777777778 0.3684210526 0.5000000000        19
              negative  0.7073170732 0.7631578947 0.7341772152        38
               neutral  0.0000000000 0.0000000000 0.0000000000         3
no sentiment expressed  0.0000000000 0.0000000000 0.0000000000         0
              positive  0.74358

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarnin

In [22]:
for review, true_label, predicted_label in evaluation_output_s_food[:25]:
  print('{}\t{}\t{}'.format(review, true_label, predicted_label))

طعم خیلی خوبی داره و توضیحات کالا کاملا با خود محصول مطابقت داشت	very positive	very positive
خوش طعم بود من طعم کارامل حس نکردم سرد مصرف کنید گرمش خوشمزه نیست	positive	negative
من پسرم عاشق ایناس ولی دیگه به خاطر حفظ محیط زیست فقط زمانهایی که مجبور باشم شیر دونه ای میخرم و سعی میکنم دیگه کمتر شیر با بسته بندی تتراپک استفاده کنم	mixed	no sentiment expressed
این مدلو هم خودمون دوست داریم هم هامو کوچولوها, ظاهرش جالبه و خیلی هم توخالی نیست	very positive	positive
در شگفت انگیز گرفتم قیمت مناسبی داشت امیدوارم دیر پز نباشه	positive	positive
با. سلام من تقریبا اکثر تن ماهی های با کیفیت و خوشنام بازارو مصرف کردم به جرأت میتونم بگم طبیعت در صدر جدول کیفیت قرار میگیره.	very positive	positive
غلیظ بود با طعم وانیل، اما قیمتش خیلی بالاست و خوش مزه نبود از نظر من. توصیه نمیکنم.	very negative	negative
قبلا راضی بودم اما اینبار واقعااا بد بود و البته دیجی کالا مبلغ رو برام برگرداند	mixed	very negative
واقعا خوب بود،تازه و بدون کوچک ترین بو با قیمت مناسب	very positive	very positive
من چند تا خریدم و ب

In [23]:
output_file_name = "sentiment_analysis_food_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for review, true_label, predicted_label in evaluation_output_s_food:
    output_file.write('{}\t{}\t{}\n'.format(review, true_label, predicted_label))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Aspect-base Sentiment Analysis - movie category

In [24]:
test_reviews_movie, test_aspects_movie, test_labels_movie = sa_model.load_dataset_test_file(
    dataset_name="pasinlu-aspect-sentiment", 
    dataset_file="./parsinlu/data/sentiment-analysis/movie_test.jsonl",    
    label_map={
        '-3': 'no sentiment expressed',
        '-2': 'very negative',
        '-1': 'negative',
        '0': 'neutral',
        '1': 'positive',
        '2': 'very positive',
        '3': 'mixed'
        }
    )
print(test_reviews_movie[:5])
print(test_aspects_movie[:5])
print(test_labels_movie[:5])
print(len(test_reviews_movie))
print(len(test_aspects_movie))
print(len(test_labels_movie))

['یک فیلم با موضوع جدید و عجیب و شاید برای خیلیها غیرقابل باور ! دیدنش برام خیلی جذاب بود البته برای خیلیها هم نبود، اینکه در آخر فیلم از خودت میپرسی خب که چی؟! یکم حس سردرگمی بهت میداد و نمیزاشت همزادپنداری کنی با شخصیت اول فیلم! ولی در کل پیشنهاد میشود.', 'یک فیلم با موضوع جدید و عجیب و شاید برای خیلیها غیرقابل باور ! دیدنش برام خیلی جذاب بود البته برای خیلیها هم نبود، اینکه در آخر فیلم از خودت میپرسی خب که چی؟! یکم حس سردرگمی بهت میداد و نمیزاشت همزادپنداری کنی با شخصیت اول فیلم! ولی در کل پیشنهاد میشود.', 'یک فیلم با موضوع جدید و عجیب و شاید برای خیلیها غیرقابل باور ! دیدنش برام خیلی جذاب بود البته برای خیلیها هم نبود، اینکه در آخر فیلم از خودت میپرسی خب که چی؟! یکم حس سردرگمی بهت میداد و نمیزاشت همزادپنداری کنی با شخصیت اول فیلم! ولی در کل پیشنهاد میشود.', 'یک فیلم با موضوع جدید و عجیب و شاید برای خیلیها غیرقابل باور ! دیدنش برام خیلی جذاب بود البته برای خیلیها هم نبود، اینکه در آخر فیلم از خودت میپرسی خب که چی؟! یکم حس سردرگمی بهت میداد و نمیزاشت همزادپنداری کنی با شخصیت اول فیلم

In [25]:
!nvidia-smi
!lscpu

Mon Aug  2 13:07:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    40W /  70W |   9414MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
sa_model.mt5_aspect_sentiment_analysis_inference(test_reviews_movie[:5], test_aspects_movie[:5], device)

['no sentiment expressed',
 'very negative',
 'no sentiment expressed',
 'no sentiment expressed',
 'no sentiment expressed']

In [27]:
evaluation_output_as_movie = sa_model.mt5_aspect_sentiment_analysis_evaluation(test_reviews_movie, test_aspects_movie, test_labels_movie, device, max_length=512, batch_size=32)

#reviews:816, #aspects:816, #labels:816
#batch: 26
Start to evaluate test data ...
inference time for step 0: 4.966977403999977
inference time for step 1: 4.969669732000057
inference time for step 2: 4.965248039000016
inference time for step 3: 4.984715126999959
inference time for step 4: 4.9651783969999315
inference time for step 5: 5.006119432999981
inference time for step 6: 4.99614366600008
inference time for step 7: 5.012500208000006
inference time for step 8: 4.9931217460000425
inference time for step 9: 4.990028319999965
inference time for step 10: 4.981854529999964
inference time for step 11: 4.977666092999925
inference time for step 12: 4.988350515999969
inference time for step 13: 4.974343758000032
inference time for step 14: 4.980769368999972
inference time for step 15: 4.973837065999987
inference time for step 16: 4.977275288000101
inference time for step 17: 4.968268808000062
inference time for step 18: 4.966656169999965
inference time for step 19: 4.981381845000101
infere

In [28]:
for review, aspect, true_label, predicted_label in evaluation_output_as_movie[:25]:
  print('{}\t{}\t{}\t{}'.format(review, aspect, true_label, predicted_label))

یک فیلم با موضوع جدید و عجیب و شاید برای خیلیها غیرقابل باور ! دیدنش برام خیلی جذاب بود البته برای خیلیها هم نبود، اینکه در آخر فیلم از خودت میپرسی خب که چی؟! یکم حس سردرگمی بهت میداد و نمیزاشت همزادپنداری کنی با شخصیت اول فیلم! ولی در کل پیشنهاد میشود.	نظر شما در مورد صداگذاری و جلوه های صوتی فیلم  درساژ چیست؟	no sentiment expressed	no sentiment expressed
یک فیلم با موضوع جدید و عجیب و شاید برای خیلیها غیرقابل باور ! دیدنش برام خیلی جذاب بود البته برای خیلیها هم نبود، اینکه در آخر فیلم از خودت میپرسی خب که چی؟! یکم حس سردرگمی بهت میداد و نمیزاشت همزادپنداری کنی با شخصیت اول فیلم! ولی در کل پیشنهاد میشود.	نظر شما در مورد داستان، فیلمنامه، دیالوگ ها و موضوع فیلم  درساژ چیست؟	mixed	very negative
یک فیلم با موضوع جدید و عجیب و شاید برای خیلیها غیرقابل باور ! دیدنش برام خیلی جذاب بود البته برای خیلیها هم نبود، اینکه در آخر فیلم از خودت میپرسی خب که چی؟! یکم حس سردرگمی بهت میداد و نمیزاشت همزادپنداری کنی با شخصیت اول فیلم! ولی در کل پیشنهاد میشود.	نظر شما در مورد موسیقی فیلم  درساژ چیست؟	no

In [29]:
output_file_name = "aspect_sentiment_analysis_movie_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for review, aspect, true_label, predicted_label in evaluation_output_as_movie:
    output_file.write('{}\t{}\t{}\t{}\n'.format(review, aspect, true_label, predicted_label))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()

### Sentiment Analysis - movie category

In [30]:
all_reviews_movie, all_labels_movie = sa_model.load_dataset_file(
    dataset_name="pasinlu-review-sentiment", 
    dataset_file="./parsinlu/data/sentiment-analysis/movie.jsonl",    
    label_map={
        '-3': 'no sentiment expressed',
        '-2': 'very negative',
        '-1': 'negative',
        '0': 'neutral',
        '1': 'positive',
        '2': 'very positive',
        '3': 'mixed'
        }
    )
print(all_reviews_movie[:5])
print(all_labels_movie[:5])
print(len(all_reviews_movie))
print(len(all_labels_movie))

['یکی از دوستان اشاره خوبی داشتن   چقد موسیقی حماسی و بی مورد؟  فقط میتونن بگم این سوژه اگه به گروه و\u200c کَست بهتری داده میشه نتیجه کار خیلی قابل قبول تر از این میشد', 'مشکل اغلب این فیلم\u200cهایی که قصد انتقاد از مهاجرت و مصائب\u200cاش را دارند، در این است که بعلت کمبود منابع، امکان ادامه دادن منطقی داستان و سفر کردن به کشور مقصد را چندان نمی\u200cیابند. کلبموس هیچ کدام از ایده\u200cهایش را ادامه نمی\u200cدهد. سردستی و بی\u200cحوصله، روایت را اندکی جلو برده و ناگهان مسئله\u200cاش دچار چرخش می\u200cشود.', 'ی فیلم خوب و کار درست   تو بازار کمدیهای تکراری ی کمدی جدید واقعا جای قدر دانی داره .', 'یه فیلم خوب ...   که میشه وقت گذاشت و بی هیچ پشیمانی دید و با رضایت از سینما خارج شد     تبریک به آقای سیدی عزیز', 'واقعاً فوق العاده بود، فقط کسانی که از سر و صدای زیاد بدشون میاد اصلاً بهشون توصیه نمیشه']
['negative', 'negative', 'very positive', 'very positive', 'very positive']
506
506


just keep test samples:

In [31]:
test_subset = []
for i, review in enumerate(all_reviews_movie):
  if review in test_reviews_movie:
    test_subset.append((review, all_labels_movie[i]))
all_reviews_movie, all_labels_movie = [t[0] for t in test_subset], [t[1] for t in test_subset]
print(all_reviews_movie[:5])
print(all_labels_movie[:5])
print(len(all_reviews_movie))
print(len(all_labels_movie))

['یکی از دوستان اشاره خوبی داشتن   چقد موسیقی حماسی و بی مورد؟  فقط میتونن بگم این سوژه اگه به گروه و\u200c کَست بهتری داده میشه نتیجه کار خیلی قابل قبول تر از این میشد', 'واقعاً فوق العاده بود، فقط کسانی که از سر و صدای زیاد بدشون میاد اصلاً بهشون توصیه نمیشه', 'امروز بالاخره فیلم رو دیدم،من واقعا در تعجبم چرا مردم ما از موضوعاتی مثل خشونت،فقر،بیچارگی و درموندگی انقدر استقبال میکنن؟! این همه کف و سوت رو جدا نمیفهمیدم... اصلا فیلم مورد انتظارم نبود .. بهترینِ امسال بدون شک تنگه ابوقریب بود و بس ..', 'این داستان پایانِ باز، برای سینمای ایران هم داره خطرناک میشه:  پایانِ باز، مث دری بود که یه کم لاش بازه،کم کم داره به یه دری تبدیل میشه که کلّا هر دولنگش تا بیخ بازه!!!', 'موضوع فیلم شاید نسبت به فیلم\u200cهایی که این سال\u200cها دیدم، جدید بود اما بازی هنرپیشه\u200cها و در کل موضوع به خاطر افراط و بزرگ\u200e\u200cنمایی بیش از حد، فیلم رو تبدیل به یک ساخته سطحی کرده.']
['negative', 'very positive', 'very negative', 'negative', 'negative']
93
93


In [32]:
!nvidia-smi
!lscpu

Mon Aug  2 13:09:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    43W /  70W |   9238MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [33]:
sa_model.mt5_sentiment_analysis_inference(all_reviews_movie[:5], device)

['mixed', 'very positive', 'very negative', 'negative', 'negative']

In [34]:
evaluation_output_s_movie = sa_model.mt5_sentiment_analysis_evaluation(all_reviews_movie, all_labels_movie, device, max_length=512, batch_size=32)

#reviews:93, #labels:93
#batch: 3
Start to evaluate test data ...
inference time for step 0: 4.9489852099999325
inference time for step 1: 4.9540873140000485
inference time for step 2: 4.544534453000097
total inference time: 14.447606977000078
total inference time / #samples: 0.15535061265591482
Test Accuracy: 0.6236559139784946
Test Precision: 0.6578502143018272
Test Recall: 0.6236559139784946
Test F1-Score(weighted average): 0.6207354506961117
Test classification Report:
                        precision    recall  f1-score   support

                 mixed  0.3333333333 0.5000000000 0.4000000000        12
              negative  0.5833333333 0.4666666667 0.5185185185        15
               neutral  0.0000000000 0.0000000000 0.0000000000         5
no sentiment expressed  0.0000000000 0.0000000000 0.0000000000         0
              positive  0.7500000000 0.3333333333 0.4615384615        18
         very negative  0.8181818182 0.9473684211 0.8780487805        19
         very posit

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarnin

In [35]:
for review, true_label, predicted_label in evaluation_output_s_movie[:25]:
  print('{}\t{}\t{}'.format(review, true_label, predicted_label))

یکی از دوستان اشاره خوبی داشتن   چقد موسیقی حماسی و بی مورد؟  فقط میتونن بگم این سوژه اگه به گروه و‌ کَست بهتری داده میشه نتیجه کار خیلی قابل قبول تر از این میشد	negative	mixed
واقعاً فوق العاده بود، فقط کسانی که از سر و صدای زیاد بدشون میاد اصلاً بهشون توصیه نمیشه	very positive	very positive
امروز بالاخره فیلم رو دیدم،من واقعا در تعجبم چرا مردم ما از موضوعاتی مثل خشونت،فقر،بیچارگی و درموندگی انقدر استقبال میکنن؟! این همه کف و سوت رو جدا نمیفهمیدم... اصلا فیلم مورد انتظارم نبود .. بهترینِ امسال بدون شک تنگه ابوقریب بود و بس ..	very negative	very negative
این داستان پایانِ باز، برای سینمای ایران هم داره خطرناک میشه:  پایانِ باز، مث دری بود که یه کم لاش بازه،کم کم داره به یه دری تبدیل میشه که کلّا هر دولنگش تا بیخ بازه!!!	negative	negative
موضوع فیلم شاید نسبت به فیلم‌هایی که این سال‌ها دیدم، جدید بود اما بازی هنرپیشه‌ها و در کل موضوع به خاطر افراط و بزرگ‎‌نمایی بیش از حد، فیلم رو تبدیل به یک ساخته سطحی کرده.	negative	negative
امتیاز: 2  بعد از خوب بد جلف بعضی از دوستان می گفتند: فقط خند

In [36]:
output_file_name = "sentiment_analysis_movie_testset_{}_outputs.txt".format(model_name.replace('/','-'))
with open(output_file_name, "w", encoding='utf8') as output_file:
  for review, true_label, predicted_label in evaluation_output_s_movie:
    output_file.write('{}\t{}\t{}\n'.format(review, true_label, predicted_label))
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
upload = drive.CreateFile({'title': output_file_name})
upload.SetContentFile(output_file_name)
upload.Upload()